### ETL

In [2]:
# Importamos las librerias a utilizar
import pandas as pd
import numpy as np
import ast


In [3]:
# Abrimos el csv con pandas para su posterior tratamiento y visualización
movies = pd.read_csv('movies_dataset.csv')
movies.head(3)


C:\Users\ABANIBI\AppData\Local\Temp\ipykernel_12196\3806792976.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('movies_dataset.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [4]:
#Verificamos información sobre las columnas y sus tipos de datos, así como la cantidad de valores no nulos en cada columna.
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

### Transformaciones:

Algunos campos, como belongs_to_collection, production_companies y otros 
(ver diccionario de datos) están anidados, esto es o bien tienen un diccionario
o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos 
al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera 
de acceder a esos datos sin desanidarlos.

In [5]:
#Verifica nulos en la columna [belongs_to_collection]
movies.belongs_to_collection.isnull().sum() 

40972

In [6]:
def fetch_name(obj): #convierte una cadena que representa una lista de diccionarios en una lista de nombres de los valores asociados a una clave en los diccionarios.
    try:
        L=[]
        for i in ast.literal_eval(obj):
            L.append(i['name']);
        return L
    except (ValueError, TypeError):
        return []

def fetch_name2(obj):  #extrae el valor asociado a la clave 'name' de un diccionario que está representado como una cadena.
    
    if isinstance(obj, str) and '{' in obj:
        dic = ast.literal_eval(obj)

        return dic['name']



movies['genres'] = movies['genres'].apply(fetch_name) #La función convierte la lista de diccionarios en una lista de los nombres de los géneros.
movies['belongs_to_collection'] = movies['belongs_to_collection'].apply(fetch_name2) #La función extrae el nombre de la colección del diccionario y lo guarda en una nueva columna.
movies['production_companies']  = movies['production_companies'].apply(fetch_name)# La función convierte la lista de diccionarios en una lista de los nombres de las compañías.
movies['production_countries']  = movies['production_countries'].apply(fetch_name)#  La función convierte la lista de diccionarios en una lista de los nombres de los países.
movies['spoken_languages']      = movies['spoken_languages'].apply(fetch_name)# La función convierte la lista de diccionarios en una lista de los nombres de los idiomas.

In [7]:
# Reemplazamos los None por 'Sin dato'
#movies['belongs_to_collection'].fillna('Sin dato', inplace=True)


In [8]:
#Verificamos la columna 'belongs_to_collection'
movies['belongs_to_collection']

0                  Toy Story Collection
1                                  None
2             Grumpy Old Men Collection
3                                  None
4        Father of the Bride Collection
                      ...              
45461                              None
45462                              None
45463                              None
45464                              None
45465                              None
Name: belongs_to_collection, Length: 45466, dtype: object

Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

In [9]:
#Verificamos valores nulos de la columna 'revenue'
movies['revenue'].isnull().sum()

6

In [10]:
#Verificamos valores nulos de la columna 'budget'
movies['budget'].isnull().sum()

0

In [11]:
# Reemplazamos los nulos por 0, en este caso solo la columna ´revenue'
movies['revenue'] = movies['revenue'].fillna(0)

Los valores nulos del campo release date deben eliminarse.

In [12]:
#Verificamos datos nulos en la columna 'release_date'
movies['release_date'].isnull().sum()

87

In [13]:
# Eliminamos los valores nulos en la columna 'release_date'
movies = movies.dropna(subset=['release_date'])



De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [14]:
#Convierte la columna "release_date" en una columna de fecha y asegura que los valores que no se pueden convertir sean reemplazados por valores nulos.
movies["release_date"] = pd.to_datetime(movies["release_date"], errors="coerce")


In [15]:
#Verificamos la columna 'release_date'
movies['release_date']

0       1995-10-30
1       1995-12-15
2       1995-12-22
3       1995-12-22
4       1995-02-10
           ...    
45460   1991-05-13
45462   2011-11-17
45463   2003-08-01
45464   1917-10-21
45465   2017-06-09
Name: release_date, Length: 45379, dtype: datetime64[ns]

In [16]:
#Creamos una nueva columna 'release_year' y extraemos el año de la columna
movies['release_year'] = movies['release_date'].dt.year
movies["release_year"] = pd.to_datetime(movies["release_date"], errors="coerce")
movies['release_year'] = movies['release_year'].dt.strftime('%Y')

Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [17]:
movies['budget'].describe() # Devuelve un resumen estadístico de la columna "budget" 

count     45379
unique     1226
top           0
freq      36490
Name: budget, dtype: object

In [18]:
# Convierte valores numéricos 
movies['budget'] = pd.to_numeric(movies['budget'], errors='coerce')

In [19]:
# Función para calcular el retorno de inversión
def calculate_return(row):
    revenue = row['revenue']
    budget = row['budget']
    if pd.isnull(revenue) or pd.isnull(budget) or budget == 0:
        return 0
    return revenue / budget

# Se crea la nueva columna 'return' y aplica la función calculte_return a cada fila del dataframe
movies['return'] = movies.apply(calculate_return, axis=1)


In [20]:
# Verificamos la columna 'return'
movies['return']

0        12.451801
1         4.043035
2         0.000000
3         5.090760
4         0.000000
           ...    
45460     0.000000
45462     0.000000
45463     0.000000
45464     0.000000
45465     0.000000
Name: return, Length: 45379, dtype: float64

Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,vote_count,poster_path y homepage.

In [21]:
#Verificamos las columnas de dataset
movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'release_year', 'return'],
      dtype='object')

In [22]:
#Eliminamos las columnas que nos asignaron
movies = movies.drop(['video', 'imdb_id', 'adult', 'original_title', 'vote_count', 'poster_path', 'homepage' ], axis=1)


Crear 6 funciones para los endpoints que se consumirán en la API

In [23]:
#Agrega una nueva columna llamada 'month' y extrae el nombre del mes de cada fecha
movies['month'] = movies['release_date'].dt.month_name()

In [24]:
#Verificamos la columna 'month'
movies.month

0         October
1        December
2        December
3        December
4        February
           ...   
45460         May
45462    November
45463      August
45464     October
45465        June
Name: month, Length: 45379, dtype: object

In [25]:
# Obtenemos el número de películas lanzadas en un mes específico
def peliculas_mes(mes):
    mes = mes.capitalize() #convierte el primer carácter de la cadena a mayúscula y el resto de los caracteres a minúscula.
    cantidad = movies.loc[movies['month'] == mes, 'title'].count()
    return {'mes':mes.lower(), 'cantidad':cantidad}

In [26]:
#Verificamos los valores únicos de la columna 
movies['month'].unique()

array(['October', 'December', 'February', 'November', 'September', 'May',
       'April', 'August', 'July', 'June', 'January', 'March', nan],
      dtype=object)

In [27]:
#Reemplazamos el valor de los meses en inglés por el nombre en español
movies['month'] = movies['month'].replace('January', 'Enero')
movies['month'] = movies['month'].replace('February', 'Febrero')
movies['month'] = movies['month'].replace('March', 'Marzo')
movies['month'] = movies['month'].replace('April', 'Abril')
movies['month'] = movies['month'].replace('May', 'Mayo')
movies['month'] = movies['month'].replace('June', 'Junio')
movies['month'] = movies['month'].replace('July', 'Julio')
movies['month'] = movies['month'].replace('August', 'Agosto')
movies['month'] = movies['month'].replace('September', 'Setiembre')
movies['month'] = movies['month'].replace('October', 'Octubre')
movies['month'] = movies['month'].replace('November', 'Noviembre')
movies['month'] = movies['month'].replace('December', 'Diciembre')

1. def peliculas_mes(mes): '''Se ingresa el mes y la funcion retorna la cantidad de peliculas que se estrenaron ese mes (nombre del mes, en str, ejemplo 'enero') historicamente''' return {'mes':mes, 'cantidad':respuesta}

In [28]:
peliculas_mes('AGOSTO')

{'mes': 'agosto', 'cantidad': 3394}

2. def peliculas_dia(dia): '''Se ingresa el dia y la funcion retorna la cantidad de peliculas que se estrenaron ese dia (de la semana, en str, ejemplo 'lunes') historicamente''' return {'dia':dia, 'cantidad':respuesta}

In [29]:
#Crea una nueva columna 'weekday' que contiene el nombre del día de la semana 
movies['weekday'] = movies['release_date'].dt.strftime('%A')

In [30]:
#Verificamos valores únicos
movies['weekday'].unique()

array(['Monday', 'Friday', 'Thursday', 'Wednesday', 'Saturday', 'Tuesday',
       'Sunday', nan], dtype=object)

In [31]:
#Reemplazamos el valor de los nombres de la semana en inglés por el nombre en español
movies['weekday'] = movies['weekday'].replace('Monday', 'Lunes')
movies['weekday'] = movies['weekday'].replace('Tuesday', 'Martes')
movies['weekday'] = movies['weekday'].replace('Wednesday', 'Miercoles')
movies['weekday'] = movies['weekday'].replace('Thursday', 'Jueves')
movies['weekday'] = movies['weekday'].replace('Friday', 'Viernes')
movies['weekday'] = movies['weekday'].replace('Saturday', 'Sabado')
movies['weekday'] = movies['weekday'].replace('Sunday', 'Domingo')

In [32]:
movies.weekday #Verificamos la columna 'weekday'

0          Lunes
1        Viernes
2        Viernes
3        Viernes
4        Viernes
          ...   
45460      Lunes
45462     Jueves
45463    Viernes
45464    Domingo
45465    Viernes
Name: weekday, Length: 45379, dtype: object

In [33]:
#la función devuelve un diccionario con el nombre del día de la semana y la cantidad de películas lanzadas en ese día.
def peliculas_dia(dia):
    dia = dia.capitalize()
    cantidad = movies.loc[movies['weekday'] == dia, 'title'].count()
    return {'dia':dia, 'cantidad':cantidad}




In [34]:
peliculas_dia('Lunes')

{'dia': 'Lunes', 'cantidad': 3503}

3. def franquicia(franquicia): '''Se ingresa la franquicia, retornando la cantidad de peliculas, ganancia total y promedio''' return {'franquicia':franquicia, 'cantidad':respuesta, 'ganancia_total':respuesta, 'ganancia_promedio':respuesta}

In [35]:
#toma como entrada el nombre de una franquicia de películas y devuelve un diccionario con información relevante de la franquicia, 
def franquicia_info(franquicia):
    cantidad_peliculas = (movies[movies['belongs_to_collection'] == franquicia]).shape[0]
    df_presupuesto = (movies[movies['belongs_to_collection'] == franquicia])['budget']
    df_ingresos = (movies[movies['belongs_to_collection'] == franquicia])['revenue']
    ganancia_total = df_ingresos.sum() - df_presupuesto.sum()
    ganancia_promedio = ganancia_total/cantidad_peliculas
    return{'franquicia':franquicia , 'cantidad':cantidad_peliculas, 'ganancia_total':ganancia_total, 'ganancia_promedio':ganancia_promedio}

In [36]:
franquicia_info('Toy Story Collection')

{'franquicia': 'Toy Story Collection',
 'cantidad': 3,
 'ganancia_total': 1617890605.0,
 'ganancia_promedio': 539296868.3333334}

4. def peliculas_pais(pais): '''Ingresas el pais, retornando la cantidad de peliculas producidas en el mismo''' return {'pais':pais, 'cantidad':respuesta}

In [37]:
#Esta función busca el número de películas que fueron producidas en un país específico.
def peliculas_pais(pais):
    pais = pais.lower()
    cantidad = movies[movies['production_countries'].apply(lambda x: pais in [i.lower() for i in x])]['title'].count()
    return {'pais':pais, 'cantidad':cantidad}

In [38]:
peliculas_pais('Philippines')

{'pais': 'philippines', 'cantidad': 83}

In [39]:
peliculas_pais('United States of America')

{'pais': 'united states of america', 'cantidad': 21147}

5. def productoras(productora): '''Ingresas la productora, retornando la ganancia total y la cantidad de peliculas que produjeron''' return {'productora':productora, 'ganancia_total':respuesta, 'cantidad':respuesta}

In [40]:
#Selecciona las películas que fueron producidas, calcula la cantidad de películas y calcula la ganancia total sumando los ingresos y restando los presupuestos, y devuelve los resultados en un diccionario.
def productoras(productora):
    productora = productora.lower()
    df = movies[movies['production_companies'].apply(lambda x: productora in [i.lower() for i in x])]
    cantidad_peliculas = df.shape[0]
    df_presupuesto = df['budget']
    df_ingresos = df['revenue']
    ganancia_total = df_ingresos.sum() - df_presupuesto.sum()
    return{'productora':productora , 'ganancia_total':ganancia_total, 'cantidad':cantidad_peliculas}


In [41]:
productoras('Warner Bros.')

{'productora': 'warner bros.',
 'ganancia_total': 39967916117.0,
 'cantidad': 1250}

6. def retorno(pelicula): '''Ingresas la pelicula, retornando la inversion, la ganancia, el retorno y el año en el que se lanzo''' return {'pelicula':pelicula, 'inversion':respuesta, 'ganacia':respuesta,'retorno':respuesta, 'anio':respuesta}

In [42]:
#La función selecciona las películas que fueron producidas por la productora, calcula la cantidad de películas y crea dos nuevas Series para el presupuesto y los ingresos
#y al final calcula la ganancia
def retorno(pelicula):
    df = movies[movies['title'] == pelicula]
    inversion = df['budget'].sum()
    ganancia = df['revenue'].sum()
    retorno = ganancia / inversion
    año = df['release_year'].iloc[0]
    return {'pelicula':pelicula, 'inversion':inversion, 'ganancia':ganancia, 'retorno':retorno, 'año':año}

In [43]:
retorno('Toy Story')

{'pelicula': 'Toy Story',
 'inversion': 30000000.0,
 'ganancia': 373554033.0,
 'retorno': 12.4518011,
 'año': '1995'}

In [44]:
#Guardamos nuestra data en un csv
movies.to_csv('data_movies.csv',index=False)